In [1]:
import Base: exp, sin, cos, +, -, *, /, sqrt, convert, promote_rule, zero
import Test: @test, @testset

This is a bit different than we had in class: we are saying that `T` must be a sub-type of `Number` (actually, if you really want to bend your brain, you can sensibly define derivatives of [regular expressions](http://www.ccs.neu.edu/home/turon/re-deriv.pdf), but that's a bridge too far, even for me!); we are also stating that our new type, `Infinitesimal{T}` is also a subtype of `Number` (this is required for some arithmetic promotion to function automatically).

In [2]:
struct Infinitesimal{T <: Number} <: Number
    x::T
    dx::T
end

What follows are some rules for converting things to infinitesimals and also promoting infinitesimals to a common type; this simplifies algebra operators like `+`.  If we define the promotion and conversion right, we only need to define the `+` operation on two infinitesimals of the same type; there are automatic rules that are employed for mixed operations that try to promote and convert them to a common type by default, and these will land us in the "sum two infinitesimals" case automatically when we write `(x,dx) + y`.

If you want to read more about these rules for conversion and promotion, see [the chapter of the Julia manual](https://docs.julialang.org/en/v1/manual/conversion-and-promotion/).

First conversion.  The first method is saying how to convert an `x` of type `T` to `Infinitesimal{T}`: we just store `x` in the value slot, and zero (of the same type as x) in the infinitesimal spot.  The second method says that to convert an `Infinitesimal` storing type `S`, to one storing type `T`, we just convert the value part and infinitesimal parts to type `T`.

In [3]:
function convert(::Type{Infinitesimal{T}}, x::T) where {T}
   Infinitesimal(x, zero(x)) 
end
function convert(::Type{Infinitesimal{T}}, x::Infinitesimal{S}) where {S, T}
   Infinitesimal(T(x.x), T(x.dx))
end
# This is needed according to an error before
function convert(::Type{Infinitesimal{T}}, x::T) where {T <: Number}
    Infinitesimal(x, zero(x))
end
function convert(::Type{Infinitesimal{T}}, x::S) where {T, S <: Number}
    x_as_T = convert(T, x)
    Infinitesimal(x_as_T, zero(x_as_T))
end

convert (generic function with 193 methods)

The way we have defined `convert` would actually permit nesting of infinitesimals, as long as we define `zero` properly:

In [4]:
function zero(x::Infinitesimal{T}) where T
    Infinitesimal(zero(x.x), zero(x.dx))
end

zero (generic function with 16 methods)

Now for promotion. 

In [19]:
function promote_rule(::Type{Infinitesimal{T}}, ::Type{Infinitesimal{S}}) where {T,S}
    Infinitesimal{promote_type(T,S)}
end
function promote_rule(::Type{Infinitesimal{T}}, ::Type{S}) where {T, S <: Number}
    Infinitesimal{promote_type(T,S)}
end
function promote_rule(::Type{T}, ::Type{Infinitesimal{S}}) where {T <: Number, S}
    Infinitesimal{promote_type(T,S)}
end
function promote_rule(::Type{S}, ::Type{Infinitesimal{T}}) where {S <: AbstractIrrational, T}
    Infinitesimal{promote_type(S,T)}
end

promote_rule (generic function with 141 methods)

Now something we didn't do in class: let's define an `extract_derivative` method that can operate on scalars or arrays.

In [6]:
function extract_derivative(xdx::Infinitesimal)
    return xdx.dx
end
function extract_derivative(xs::Array)
    [extract_derivative(x) for x in xs]
end
function extract_derivative(xs::Tuple)
    convert(Tuple, [extract_derivative(x) for x in xs])
end

extract_derivative (generic function with 3 methods)

Now we break the logical flow a bit, and define the differential operator, in two cases (one single-arg differential, one multiple-arg differential).  Note the docstring-like comment (remember from our best-practices Python lecture); see best practices for [Julia docstrings](https://docs.julialang.org/en/v1/manual/documentation/).

In [29]:
"""    D([i], f)

Differential operator; with an optional integer, specifies a partial derivative 
with respect to argment `i` (from 1).

Returns a function that computes the derivative of `f`.  NOTE: at the moment, 
nested derivatives won't work: `D(D(f))` will not compute a second derivative.
"""
function D(f)
    function df(x)
        xdx = Infinitesimal(x, one(x))
        result = f(xdx)
        return extract_derivative(result)
    end
    df
end
function D(i::Integer, f)
    function df(xs...)
        xarr = [(j != i ? x : Infinitesimal(x, one(x))) for (j,x) in enumerate(xs)]
        result = f(xarr...)
        return extract_derivative(result)
    end
    df
end

D (generic function with 2 methods)

Let's look at our pretty docstring using the "help" operator:

In [8]:
?D

search: D do div Docs Dims Dict dump diff divrem digits detach DataType display



```
D([i], f)
```

Differential operator; with an optional integer, specifies a partial derivative  with respect to argment `i` (from 1).

Returns a function that computes the derivative of `f`.  NOTE: at the moment,  nested derivatives won't work: `D(D(f))` will not compute a second derivative.


Now on to some mathematical definitions.  We begin with arithmetic:

In [9]:
function +(x::Infinitesimal, y::Infinitesimal)
    Infinitesimal(x.x+y.x, x.dx+y.dx)
end
function -(x::Infinitesimal, y::Infinitesimal)
    Infinitesimal(x.x-y.x, x.dx-y.dx)
end
function -(x::Infinitesimal)
    Infinitesimal(-x.x, -x.dx)
end
function *(x::Infinitesimal, y::Infinitesimal)
    Infinitesimal(x.x*y.x, x.x*y.dx + x.dx*y.x)
end
function /(x::Infinitesimal, y::Infinitesimal)
    Infinitesimal(x.x/y.x, x.dx/y.x - x.x*y.dx/(y.x*y.x))
end

/ (generic function with 103 methods)

Now square roots:

In [10]:
function sqrt(x::Infinitesimal)
    Infinitesimal(sqrt(x.x), x.dx/(2*sqrt(x.x)))
end

sqrt (generic function with 20 methods)

And transcendental functions:

In [11]:
function exp(x::Infinitesimal)
    return Infinitesimal(exp(x.x), exp(x.x)*x.dx)
end
function sin(x::Infinitesimal)
    return Infinitesimal(sin(x.x), cos(x.x)*x.dx)
end
function cos(x::Infinitesimal)
    return Infinitesimal(cos(x.x), -sin(x.x)*x.dx)
end

cos (generic function with 13 methods)

A quick test (`exp` is its own derivative):

In [12]:
@test begin
    x = randn()
    isapprox(D(exp)(x), exp(x))
end

Test Passed

It (now---thanks to conversion and promotion) works with other types of numbers, too:

In [13]:
@testset "It works on different types" begin
    @test isapprox(D(sin)(2//3), cos(2//3)) # Rationals
    @test isapprox(D(sin)(1+2im), cos(1+2im)) # Complex integers!
    @test isapprox(D(cos)(1.0+3.5im), -sin(1.0+3.5im)) # Boring complex floating point numbers
    @test isapprox(D(sqrt)(2), 1/(2*sqrt(2.0))) # Automatically promotes to the correct type.
end

Test Summary:               | Pass  Total
It works on different types |    4      4


Test.DefaultTestSet("It works on different types", Any[], 4, false)

Here is the example from class today, which is much cleaner now that we've got the type promotion working!

In [22]:
freq = sqrt(2)

function f(x)
    return exp(-x/pi)*sin(2.0*pi*freq*x)
end
fprime = D(f)

function laborious_fprime(x)
    return exp(-x/pi)*(2*pi*freq*cos(2*pi*freq*x) - sin(2*pi*freq*x)/pi)
end

@test isapprox(fprime(3), laborious_fprime(3))

Test Passed

Let's try the partial derivative example that didn't work in class today.

In [33]:
function linear(x, m, b)
    return m*x+b
end
dldx = D(1, linear)
dldslope = D(2, linear)
dldintercept = D(3, linear)

@testset "partials of linear function" begin
    x = randn()
    m = randn()
    b = randn()
    @test isapprox(dldx(x,m,b), m)
    @test isapprox(dldslope(x,m,b), x)
    @test isapprox(dldintercept(x,m,b), 1)
end

Test Summary:               | Pass  Total
partials of linear function |    3      3


Test.DefaultTestSet("partials of linear function", Any[], 3, false)